# Classification with neural nets; convolutional neural nets

So far, we've focused on solving regression problems with neural nets.  Today, we'll look at how neural nets can help us solve categorical classification problems.  Then, we'll introduce a new kind of layer we can add to our networks called a _convolutional layer_.  Convolutional layers are in many ways similar to the _fully-connected_ layers we've been using so far, but the neuron connections and parameters are architected differently.

## 1. Classification with neural nets

To solve classification problems with neural nets, we only need to make a few changes to our network and model-fitting process.  Here are the key ideas:

* For binary classification, we still only need a single output neuron.
* For classification with more than two categories, we use one output neuron per category.
* We add an appropriate transfer/activation function to the _output_ layer of our network so we can interpret the results and calculate the loss.
* Use _cross-entropy_ for the loss.

Let's see all of this in action.

### a. Preliminaries

For this notebook, we'll be working with a classic image dataset called _[MNIST](http://yann.lecun.com/exdb/mnist/)_.  MNIST contains 28x28 grayscale images of hand-written digits (i.e., the characters 0-9).  Our task is to predict the digit that corresponds with each image.  Thus, we have a classification problem with 10 categories.

In [ ]:
import numpy as np
import pandas as pd

import plotnine as pn
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
from torch import nn
from skorch import NeuralNetClassifier
from skorch.dataset import CVSplit

We can use scikit-learn's `fetch_openml()` to download the MNIST data.

In [ ]:
mnist = fetch_openml('mnist_784', cache=False, as_frame=False)

In [ ]:
x = mnist.data.astype(np.float32)
y = mnist.target.astype(np.int)

Let's explore the data a bit to see what we have.

**Exercise:** Do you see anything that concerns you about the data?  If so, how can we fix it?

Let's take a look at a few of the images just to see what we're working with.  Our image data are arranged in 1-dimensional arrays, though, so we'll need to reshape them into 28x28 matrices to be able to view them as images.

In [ ]:
img_num = 12

img_mat = x[img_num].reshape(28, 28)
plt.imshow(img_mat, cmap='gray_r')
print(y[img_num])

### b. Define the network architecture

Our next step is to define the layers of neurons in our network.  Recall that, because we are modeling a classification problem with 10 categories, we need 10 neurons in our output layer.

**Exercise:** Modify the network below so that it can be used for classifying MNIST image data.  The only rigid requirements are the number of inputs each neuron in the first layer must accept (hint: what is the size of each image vector?) and the number of outputs in the last layer (hint: see the paragraph above).  Otherwise, feel free to modify it however you like by changing the numbers of neurons in the hidden layers or the number of layers.

In [ ]:
class MnistNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(7, 40),  # 40 neurons that each accept 7 inputs
            nn.ReLU(),
            nn.Linear(40, 20), # 20 neurons that each accept 40 inputs
            nn.ReLU(),
            nn.Linear(20, 1)    # 1 neuron that accepts 20 inputs
        )
    
    def forward(self, X):
        y = self.layers(X)
        
        return y

### c. Add a transfer/activation function to the output layer

For a binary classifier, we can use a single output node with the $sigmoid()$ transfer function we discussed earlier, which will guarantee that the output is in the range $(0.0, 1.0)$.  We can then interpret the output as the "probability" (really more of a confidence score) that the input belongs to a particular category.

For a classifier with multiple outputs, we use the _softmax()_ function.  Given a vector $\boldsymbol{z} = [z_1, z_2, ..., z_P]$, we define

$$ softmax(z_i) = \frac {e^{z_i}} {\sum_{j=1}^P e^{z_j}} .$$

In our context, $\boldsymbol{z}$ represents the output of our final layer of neurons _before_ we apply $softmax()$.

Here is a Python function that implements $softmax()$.

In [ ]:
def softmax(z):
    """
    Implements the standard softmax().
    
    z: A numpy array of floating-point numbers.
    """
    sm_val = np.exp(z) / np.exp(z).sum()
    
    return sm_val

**Exercise:** Using the `softmax()` function defined above, experiment with different input vectors and answer the following questions (you can also answer these directly by inspecting the mathematical definition above):

1. The domain (allowable input values) of $softmax()$ is any real vector (i.e., $\boldsymbol{z} \in \mathbb{R}^N$).  What is the _range_ of the output values of $softmax()$?  Put another way, what are the limits, if any, of the output values of $softmax()$?
2. The sum of the output values of $softmax()$ has an important property.  What is it?

In [ ]:
z = np.array([-1, 2, 3, 4])

softmax(z)

Our next step is to compute $softmax()$ on the outputs of our network, so let's add that to the definition of our neural net, above.

### d. Change the loss function and train the network

As mentioned above, we generally want to use _cross-entropy_ loss when training a classifier network.  The formula for cross-entropy is not particularly complicated, but the reasoning behind it is not very obvious (to me, anyway), and a thorough explanation requires some additional mathematical concepts (most notably from information theory) that I won't attempt to cover here.  Instead, here are two things you can think about:

1. Much like we used _squared error_ to measure how far a model's prediction was from the "truth", cross-entropy also gives us a measure of the "distance" between a prediction and the truth.
2. Just as we used _mean squared error_ to calculate the average squared error across an entire training dataset, we can also calculate the average cross-entropy loss across our entire training dataset and use that to update our model's parameters.

Also, we shouldn't forget to hold back a _test dataset_, independent of the _validation dataset_, that we can use to evaluate our model after training.

**Exercise:** Train your model and see what happens!  Tune the hyperparameters to get the best model you can, paying attention to everything we've discussed (e.g., validation loss, the training/validation loss curves, etc.).

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

nn_model = NeuralNetClassifier(
    MnistNet,
    train_split=CVSplit(0.25),
    batch_size=64,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    max_epochs=10,
    lr=0.0001,
    optimizer__momentum=0.9
)

nn_model.fit(x_train, y_train)

Let's visualize the loss to see how we're doing.  We'll use the same graphing function we developed in the previous notebook.

In [ ]:
def graph_loss(tr_history, ymin=0, ymax=None):
    tr_df = pd.DataFrame(tr_history)

    if ymax is None:
        ymax = tr_df.train_loss.max()
        
    g = (pn.ggplot(tr_df, pn.aes(x='epoch', y='train_loss')) +
         pn.geom_line() +
         pn.geom_line(mapping=pn.aes(y='valid_loss'), color='red') +
         pn.ylim(ymin, ymax) + pn.ylab('Loss')
    )
    
    return g

In [ ]:
graph_loss(nn_model.history, 1, 3)

### e. Evaluate the model

Finally, we can evaluate the performance of our model on the test dataset.

**Exercise:**  Experiment with the architecture of your network.  For example, how many neurons and/or layers are required to obtain good results?

## 2. Convolutional neural networks

Next, we'll look at one of the most important ideas in modern deep learning: _convolutional neural networks_.  Here are the two most important features of a convolutional layer.

* Each neuron in the layer is only connected to a subset of the outputs of the layer below.
* The neurons within a convolutional layer typically share a single set of weight parameters.  The input to the layer is therefore _convolved_ with this weight matrix; hence the name of the layer (and network).

Why (and when) is this architectural change beneficial?  Well, for one thing, think about how many parameters are required for the network we built above.  How many parameters will be required if the input image size doubles?  How does this compare to the convolutional architecture?

Another key advantage of convolutional layers is that they can take advantage of the spatial structure of an image, something that our network above cannot so easily do.

The weight sharing in a convolutional layer also means that input information is interpreted similarly across the input variables.  This can be advantageous for some kinds of data; less so for others!

Okay, that all said, let's convert our MNIST network to use convolutional layers.

### a. Define the neural net

We'll start with the simplest case of a single convolutional layer so that it easier to understand how everything works.  We'll also add a _pooling layer_ to our network, which is common practice in convolutional neural nets.  Pooling layers aggregate input features and reduce the dimensionality of the data.

In [ ]:
class MnistCnn(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layers = nn.Sequential(
            # This layer has 1 input channel (grayscale image), 16 output features, and kernel size of 3.
            nn.Conv2d(1, 16, 3),
            nn.MaxPool2d(2),
            nn.ReLU(),
            
            # Convert the layer to a 1-D vector before passing to the next layer.
            nn.Flatten(),
            
            # Each neuron in this layer needs to accept width * height * num_channels inputs.
            nn.Linear(2704, 10),
            nn.Softmax(dim=1)
        )
    
    def forward(self, X):
        y = self.layers(X)
        
        return y

### b. Fit the model

Before we can fit our model, we need to reshape our input data so that it is delivered to the network as 28x28 image matrices.

In [ ]:
x_cnn = x.reshape(-1, 1, 28, 28)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_cnn, y, test_size=0.25)

nn_model = NeuralNetClassifier(
    MnistCnn,
    train_split=CVSplit(0.25),
    batch_size=64,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    max_epochs=30,
    lr=0.04,
    optimizer__momentum=0.9
)

nn_model.fit(x_train, y_train)

In [ ]:
graph_loss(nn_model.history, 1, 2)

### c. Evaluate the model

Let's see how our model performs on the test set.

In [ ]:
nn_model.score(x_test, y_test)

**Exercise:** If everything worked, the accuracy of the convnet should have been subtantially higher than our first network.  Experiment with the network architecture to see if you can design an even more accurate network.  For example, try adding a second convolutional layer after the pooling layer.